In [1]:
import weapon_data as weapons

D:\z_outsourced_programs\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
train_data, test_data = weapons.get_data(2, #num_of_categorical_param
                                         15, #num_of_numerical_param
                                         1, #ammo_feature_column_dimension
                                         debug=True) 
print(train_data.standardized_max_values)
print(train_data.standardized_min_values)
print(test_data.standardized_max_values)
print(test_data.standardized_min_values)

Feature columns to tensor variables dictionary:
_NumericColumn(key='damages_first', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='damages_last', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='dmg_distances_first', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='dmg_distances_last', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0)) = []
_NumericColumn(key='hiprecoildec', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='hiprecoilright', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='hiprecoilup', shape=(1,), default_value=None,

In [16]:
print("Training data size %i "%train_data.num_examples)
print("Test data size %i "%test_data.num_examples)
print("Sum of data %i"%(train_data.num_examples+test_data.num_examples))

Training data size 127 
Test data size 21 
Sum of data 148


In [17]:
import tensorflow as tf

def addGeneratedWeapon(generatedWeaponTensorDict, dictToAdd):
    for key, value in generatedWeaponTensorDict.items():
        if key not in dictToAdd:
            dictToAdd[key] = []
        dictToAdd[key].append(value)

def check_data(data):
    genDict = {}

    #print to see if normalized
    print(data.data[0])
    
    #check if unnormalizing and adding works
    var_init = tf.global_variables_initializer()

    batch_size = 1
    with tf.Session() as sess:
        sess.run(var_init)

        for i in range(1):
            batch = data.next_batch(batch_size, shuffle=True)
            #print("Batch size %i" %len(batch))
            data.add_new_weapons_and_restandardize_data(batch)

            
        tensorDict = [data.decode_processed_tensor(b)[0] for b in batch]
        [addGeneratedWeapon(t, genDict) for t in tensorDict]
        #debug_printDict(tensorDict[0])


    #weapons.debug_printDict(genDict)
    print(genDict)


In [18]:
check_data(train_data)
#check_data(test_data)

[-0.51827452 -0.3254131  -0.318201   -0.0976977   0.95346259 -0.63245553
 -0.48507125  2.20872539 -1.14166007 -0.70411138  2.89288215 -0.24421696
 -0.31750032  0.36041689  1.10282999  2.38607239  0.40094614 -0.32444284
 -0.32444284 -0.48507125 -0.55901699 -0.32444284 -0.40824829  1.78885438]
Current position in dataset after next_batch = 1
Added new weapon/s! New number of examples in dataset = 22
{'damages_first': ['53.613614551401426'], 'damages_last': ['38.3556783206882'], 'dmg_distances_first': ['75.54713605599119'], 'dmg_distances_last': ['90.57099260679331'], 'firemode_Automatic': ['-0.024404424085075926'], 'firemode_Semi-Automatic': ['1.054629624819075'], 'hiprecoildec': ['2.839609098089769'], 'hiprecoilright': ['0.9117310930938824'], 'hiprecoilup': ['1.48124763734925'], 'hipstandbasespreaddec': ['2.859636856282561'], 'hipstandbasespreadinc': ['0.09407940430640921'], 'hordispersion': ['-0.005292597602774535'], 'initialspeed': ['722.677123218633'], 'magsize': ['4.842667158201834'

In [19]:
batch = test_data.next_batch(1)
weapon_dict, _ = test_data.decode_processed_tensor(batch[0])


Current position in dataset after next_batch = 2


In [20]:
batch[0]

array([-0.56036671, -0.69781175, -0.64573493, -0.66719701, -1.        ,
        1.46385011, -0.47140452, -0.25455267, -0.52869078, -0.43064628,
       -0.46735032, -0.43905548, -0.30949223, -1.41468248, -0.52526861,
       -1.16150834,  0.44118029, -0.31622777, -0.31622777,  2.12132034,
       -0.61237244, -0.31622777, -0.39735971, -0.54232614])

In [21]:
weapon_dict

{'damages_first': '26.5',
 'damages_last': '13.5',
 'dmg_distances_first': '6.0',
 'dmg_distances_last': '31.329999923706055',
 'firemode_Automatic': '0.0',
 'firemode_Semi-Automatic': '1.0',
 'hiprecoildec': '5.0',
 'hiprecoilright': '0.4000000059604645',
 'hiprecoilup': '1.0',
 'hipstandbasespreaddec': '4.5',
 'hipstandbasespreadinc': '0.15000000596046448',
 'hordispersion': '0.0',
 'initialspeed': '320.0',
 'magsize': '8.0',
 'reloadempty': '2.0',
 'rof': '449.0',
 'shotspershell': '1.0',
 'type_Pistol': '1.0',
 'type_Rifle': '0.0',
 'type_Shotgun': '0.0',
 'type_Sniper Rifle': '0.0',
 'type_Submachine Gun': '0.0'}

In [22]:
prepared_for_encoding = test_data.prepare_decoded_tensor_dict_for_encoding(weapon_dict)
original, _ = test_data.encode_features_dict(prepared_for_encoding)
test = test_data.standardize_encoded_data(original[0])

Feature columns to tensor variables dictionary:
_NumericColumn(key='damages_first', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='damages_last', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='dmg_distances_first', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='dmg_distances_last', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0)) = []
_NumericColumn(key='hiprecoildec', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='hiprecoilright', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='hiprecoilup', shape=(1,), default_value=None,

In [23]:
test

array([-0.56036671, -0.69781175, -0.64573493, -0.66719701, -1.        ,
        1.46385011, -0.47140452, -0.25455267, -0.52869078, -0.43064628,
       -0.46735032, -0.43905548, -0.30949223, -1.41468248, -0.52526861,
       -1.16150834,  0.44118029, -0.31622777, -0.31622777,  2.12132034,
       -0.61237244, -0.31622777, -0.39735971, -0.54232614])

In [11]:
test_data.decode_processed_tensor(test)

({'damages_first': '26.5',
  'damages_last': '13.5',
  'dmg_distances_first': '6.0',
  'dmg_distances_last': '31.329999923706055',
  'firemode_Automatic': '0.0',
  'firemode_Semi-Automatic': '1.0',
  'hiprecoildec': '5.0',
  'hiprecoilright': '0.4000000059604645',
  'hiprecoilup': '1.0',
  'hipstandbasespreaddec': '4.5',
  'hipstandbasespreadinc': '0.15000000596046448',
  'hordispersion': '0.0',
  'initialspeed': '320.0',
  'magsize': '8.0',
  'reloadempty': '2.0',
  'rof': '449.0',
  'shotspershell': '1.0',
  'type_Pistol': '1.0',
  'type_Rifle': '0.0',
  'type_Shotgun': '0.0',
  'type_Sniper Rifle': '0.0',
  'type_Submachine Gun': '0.0'},
 array([2.65000000e+01, 1.35000000e+01, 6.00000000e+00, 3.13299999e+01,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 5.00000000e+00,
        4.00000006e-01, 1.00000000e+00, 4.50000000e+00, 1.50000006e-01,
        0.00000000e+00, 3.20000000e+02, 8.00000000e+00, 2.00000000e+00,
        4.49000000e+02, 1.00000000e+00, 0.00000000e+00, 1.00000